In [ ]:
import cv2
import numpy as np

In [ ]:
def display_image(img):
    cv2.imshow('image', img)  # Display the image
    cv2.waitKey(0)  # Wait for any key to be pressed (with the image window active)
    cv2.destroyAllWindows()  # Close all windows

In [ ]:
def processing(img, skip_dilate=False):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Note that kernel sizes must be positive and odd and the kernel must be square.
    process = cv2.GaussianBlur(img.copy(), (9, 9), 0)

    # cv2.adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, constant(c)) blockSize – Size of
    # a pixel neighborhood that is used to calculate a threshold value for the pixel: 3, 5, 7, and so on. C –
    # Constant subtracted from the mean or weighted mean (see the details below). Normally, it is positive but may be
    # zero or negative as well.
    process = cv2.adaptiveThreshold(process, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # we need grid edges so we need to invert colors
    process = cv2.bitwise_not(process, process)

    if not skip_dilate:
        # This is only used for sudoku processing and not for cell processing
        # np.uint8 will wrap.
        # For example, 235+30 = 9.
        kernel = np.array([[0., 1., 0.], [1., 1., 1.], [0., 1., 0.]], np.uint8)
        process = cv2.dilate(process, kernel)

    return process

In [ ]:
def find_corners(img):
    # findContours: boundaries of shapes having same intensity
    # CHAIN_APPROX_SIMPLE - stores only minimal information of points to describe contour
    # -> RETR_EXTERNAL: gives "outer" contours, so if you have (say) one contour enclosing another (like concentric circles), only the outermost is given.
    # cv2.ContourArea(): Finds area of outermost polygon(largest feature) in img.
    ext_contours = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ext_contours = ext_contours[0] if len(ext_contours) == 2 else ext_contours[1]
    ext_contours = sorted(ext_contours, key=cv2.contourArea, reverse=True)

    # loop runs only once
    for c in ext_contours:
        peri = cv2.arcLength(c, True)
        # cv2.approxPolyDP(curve, epsilon, closed[, approxCurve])
        # Curve-> hers is the largest contour
        # epsilon -> Parameter specifying the approximation accuracy. This is the maximum distance between the original curve and its approximation.
        # closed – If true, the approximated curve is closed. Otherwise, it is not closed.
        # approxPolyDP returns the approximate curve in the same type as the input curve
        approx = cv2.approxPolyDP(c, 0.015 * peri, True)
        if len(approx) == 4:
            # Here we are looking for the largest 4 sided contour
            return approx

In [ ]:
def order_corner_points(corners):
    # Corners[0],... stores in format [[x y]]
    # Separate corners into individual points
    # Index 0 - top-right
    #       1 - top-left
    #       2 - bottom-left
    #       3 - bottom-right
    corners = [(corner[0][0], corner[0][1]) for corner in corners]
    top_r, top_l, bottom_l, bottom_r = corners[0], corners[1], corners[2], corners[3]
    return top_l, top_r, bottom_r, bottom_l

In [ ]:
# Crop the image
def perspective_transform(image, corners):
    # Order points in clockwise order
    ordered_corners = order_corner_points(corners)
    top_l, top_r, bottom_r, bottom_l = ordered_corners

    # Determine width of new image which is the max distance between
    # (bottom right and bottom left) or (top right and top left) x-coordinates
    width_A = np.sqrt(((bottom_r[0] - bottom_l[0]) ** 2) + ((bottom_r[1] - bottom_l[1]) ** 2))
    width_B = np.sqrt(((top_r[0] - top_l[0]) ** 2) + ((top_r[1] - top_l[1]) ** 2))
    width = max(int(width_A), int(width_B))

    # Determine height of new image which is the max distance between
    # (top right and bottom right) or (top left and bottom left) y-coordinates
    height_A = np.sqrt(((top_r[0] - bottom_r[0]) ** 2) + ((top_r[1] - bottom_r[1]) ** 2))
    height_B = np.sqrt(((top_l[0] - bottom_l[0]) ** 2) + ((top_l[1] - bottom_l[1]) ** 2))
    height = max(int(height_A), int(height_B))

    # Construct new points to obtain top-down view of image in
    # top_r, top_l, bottom_l, bottom_r order
    dimensions = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1],
                           [0, height - 1]], dtype="float32")

    # Convert to Numpy format
    ordered_corners = np.array(ordered_corners, dtype="float32")

    # calculate the perspective transform matrix and warp
    # the perspective to grab the screen
    grid = cv2.getPerspectiveTransform(ordered_corners, dimensions)

    # Return the transformed image
    return cv2.warpPerspective(image, grid, (width, height))

In [ ]:
def create_image_grid(img):
    grid = np.copy(img)
    # not all sudoku out there have same width and height in the small squares so we need to consider differnt heights and width
    edge_h = np.shape(grid)[0]
    edge_w = np.shape(grid)[1]
    celledge_h = edge_h // 9
    celledge_w = np.shape(grid)[1] // 9

    grid = cv2.cvtColor(grid, cv2.COLOR_BGR2GRAY)

    # Adaptive thresholding the cropped grid and inverting it
    grid = cv2.bitwise_not(grid, grid)


    tempgrid = []
    for i in range(celledge_h, edge_h + 1, celledge_h):
        for j in range(celledge_w, edge_w + 1, celledge_w):
            rows = grid[i - celledge_h:i]
            tempgrid.append([rows[k][j - celledge_w:j] for k in range(len(rows))])

    # Creating the 9X9 grid of images
    finalgrid = []
    for i in range(0, len(tempgrid) - 8, 9):
        finalgrid.append(tempgrid[i:i + 9])

    # Converting all the cell images to np.array
    for i in range(9):
        for j in range(9):
            finalgrid[i][j] = np.array(finalgrid[i][j])

    try:
        for i in range(9):
            for j in range(9):
                np.os.remove("BoardCells1/cell" + str(i) + str(j) + ".jpg")
    except:
        pass
    for i in range(9):
        for j in range(9):
            cv2.imwrite(str("BoardCells1/cell" + str(i) + str(j) + ".jpg"), finalgrid[i][j])

    return finalgrid

In [ ]:
def scale_and_centre(img, size, margin=20, background=0):
    """Scales and centres an image onto a new background square."""
    h, w = img.shape[:2]

    def centre_pad(length):
        """Handles centering for a given length that may be odd or even."""
        if length % 2 == 0:
            side1 = int((size - length) / 2)
            side2 = side1
        else:
            side1 = int((size - length) / 2)
            side2 = side1 + 1
        return side1, side2

    def scale(r, x):
        return int(r * x)

    if h > w:
        t_pad = int(margin / 2)
        b_pad = t_pad
        ratio = (size - margin) / h
        w, h = scale(ratio, w), scale(ratio, h)
        l_pad, r_pad = centre_pad(w)
    else:
        l_pad = int(margin / 2)
        r_pad = l_pad
        ratio = (size - margin) / w
        w, h = scale(ratio, w), scale(ratio, h)
        t_pad, b_pad = centre_pad(h)

    img = cv2.resize(img, (w, h))
    img = cv2.copyMakeBorder(img, t_pad, b_pad, l_pad, r_pad, cv2.BORDER_CONSTANT, None, background)
    return cv2.resize(img, (size, size))


In [ ]:
def extract():
    # print("Enter image name: ")
    # image_url = input()
    img = cv2.imread('ii.jpg')
    processed_sudoku = processing(img)
    sudoku = find_corners(processed_sudoku)
    transformed = perspective_transform(img, sudoku)
    cropped = 'cropped_img.png'
    cv2.imwrite(cropped, transformed)
    transformed = cv2.resize(transformed, (450, 450))
    sudoku = create_image_grid(transformed)
    return sudoku

In [ ]:
sudoku=extract()

In [ ]:
sudoku

In [ ]:
import cv2
import numpy as np
from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
from image_processes import extract, scale_and_centre, order_corner_points

In [ ]:
def extract_number_image(img_grid):
    tmp_sudoku = [[0 for i in range(9)] for j in range(9)]
    for i in range(9):
        for j in range(9):

            image = img_grid[i][j]
            image = cv2.resize(image, (28, 28))
            original = image.copy()

            thresh = 128  # define a threshold, 128 is the middle of black and white in grey scale
            # threshold the image
            gray = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]

            # Find contours
            cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]

            for c in cnts:
                x, y, w, h = cv2.boundingRect(c)

                if (x < 3 or y < 3 or h < 3 or w < 3):
                    # Note the number is always placed in the center
                    # Since image is 28x28
                    # the number will be in the center thus x >3 and y>3
                    # Additionally any of the external lines of the sudoku will not be thicker than 3
                    continue
                ROI = gray[y:y + h, x:x + w]
                ROI = scale_and_centre(ROI, 120)
                # display_image(ROI)

                # Writing the cleaned cells
                cv2.imwrite("CleanedBoardCells/cell{}{}.png".format(i, j), ROI)
                tmp_sudoku[i][j] = predict(ROI)

    return tmp_sudoku

In [ ]:
def predict(img_grid):
    image = img_grid.copy()
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # image = cv2.threshold(image, 140, 255, cv2.THRESH_BINARY)[1]
    image = cv2.resize(image, (28, 28))
    # display_image(image)
    image = image.astype('float32')
    image = image.reshape(1, 28, 28, 1)
    image /= 255

    # plt.imshow(image.reshape(28, 28), cmap='Greys')
    # plt.show()
    model = load_model('model.h5')
    pred = model.predict(image.reshape(1, 28, 28, 1), batch_size=1)

    # print(pred.argmax())

    return pred.argmax()


In [ ]:
temp_sudoku=extract_number_image(sudoku)

In [ ]:
temp_sudoku

In [ ]:
class Solution(object):
    def solveSudoku(self, board):
        self.board=board
        self.row = [[0 for i in range(9)] for j in range(9)]
        self.col=[[0 for i in range(9)] for j in range(9)]
        self.box=[[0 for i in range(9)] for j in range(9)]
        for r in range(9):
            for c in range(9):
                if(board[r][c]!=0):
                    self.row[r][board[r][c]-1]=self.col[c][board[r][c]-1]=self.box[3*(r//3)+(c//3)][board[r][c]-1]=1
        flag=self.solve(0,0)
        if(not flag):
            print("Sorry,couldn't solve that. (:")
        return self.board
    def solve(self,i,j):
        if i==9:
            return True
        if j==9:
            return self.solve(i+1,0)
        if(self.board[i][j]!=0):
            return self.solve(i,j+1)
        for c in range(1,10):
            if(self.row[i][c-1]==0 and self.col[j][c-1]==0 and self.box[3 * (i//3) +(j//3)][c-1]==0):
                self.board[i][j]=c
                self.row[i][c-1]=1
                self.col[j][c-1]=1
                self.box[3 * (i//3) +(j//3)][c-1]=1
                if(self.solve(i,j+1)):
                    return True
                self.board[i][j]=0
                self.row[i][c-1]=0
                self.col[j][c-1]=0
                self.box[3* (i//3) +(j//3)][c-1]=0      
        return False


In [ ]:
su=[[5,3,0,0,7,0,0,0,0],[6,0,0,1,9,5,0,0,0],[0,9,8,0,0,0,0,6,0],[8,0,0,0,6,0,0,0,3],[4,0,0,8,0,3,0,0,1],[7,0,0,0,2,0,0,0,6],[0,6,0,0,0,0,2,8,0],[0,0,0,4,1,9,0,0,5],[0,0,0,0,8,0,0,7,9]]

In [ ]:
s=Solution()
s.solveSudoku(su)

In [1]:
from image_processes import extract as extract_img_grid
from predict import extract_number_image as sudoku_extracted
from solve import  solve_sudoku

def display_gameboard(sudoku):
    for i in range(len(sudoku)):
        if i % 3 == 0:
            if i == 0:
                print(" ┎─────────┰─────────┰─────────┒")
            else:
                print(" ┠─────────╂─────────╂─────────┨")

        for j in range(len(sudoku[0])):
            if j % 3 == 0:
                print(" ┃ ", end=" ")

            if j == 8:
                print(sudoku[i][j] if sudoku[i][j] != 0 else ".", " ┃")
            else:
                print(sudoku[i][j] if sudoku[i][j] != 0 else ".", end=" ")

    print(" ┖─────────┸─────────┸─────────┚")

def main():
    # Calling the image_prcoesses.py extract function to get a processed np.array of cells
    image_grid = extract_img_grid()
    print("Image Grid extracted")

    # note we have alreday created and stored the model but if you want to do that again use the following command
    # create_and_save_Model()

    # Sudoku extract
    sudoku = sudoku_extracted(image_grid)
    print("Extracted and predict digits in the Sudoku")

    print("\n\nSudoku:")
    display_gameboard(sudoku)

    print("\nSolving the Sudoku...\n")
    solvable, solved = solve_sudoku(sudoku)

    if(solvable):
        print("\nSolved Sudoku:")
        display_gameboard(solved)

    print("Program End")

if __name__ == '__main__':
    main()

Image Grid extracted


Extracted and predict digits in the Sudoku


Sudoku:
 ┎─────────┰─────────┰─────────┒
 ┃  8 . .  ┃  . 7 .  ┃  . . 9  ┃
 ┃  . 5 .  ┃  8 . 7  ┃  . 7 .  ┃
 ┃  . . 4  ┃  . 9 .  ┃  7 . .  ┃
 ┠─────────╂─────────╂─────────┨
 ┃  . 6 .  ┃  1 . 7  ┃  . 2 .  ┃
 ┃  5 . 8  ┃  . 6 .  ┃  1 . 7  ┃
 ┃  . 7 .  ┃  5 . 2  ┃  . 8 .  ┃
 ┠─────────╂─────────╂─────────┨
 ┃  . . 1  ┃  . 4 .  ┃  6 . .  ┃
 ┃  . 8 .  ┃  3 . 4  ┃  . 4 .  ┃
 ┃  3 . .  ┃  . 5 .  ┃  . . 8  ┃
 ┖─────────┸─────────┸─────────┚

Solving the Sudoku...

Program End
